# 🧠 Build a Tiny GPT from Scratch (Word-Level, WikiText-2)

In this Colab, we'll:

1. Load the **WikiText-2** dataset
2. Preprocess it using **word-level tokenization**
3. Build a **mini GPT model** with a Transformer block
4. Train it to predict the next word
5. Generate word-by-word text with it

Let's go! 🚀


In [1]:
%pip install --upgrade datasets fsspec torch

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 3.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 41.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 68.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 61.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 49.8 MB/s

## 📥 Step 1: Load and Tokenize WikiText-2

We'll use Hugging Face Datasets to load the raw WikiText-2 corpus and tokenize it word-by-word.


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from datasets import load_dataset

/home/ugrads/majors/ishamkm23/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from datasets import load_dataset

# Load wikitext-2-raw
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

# Combine all non-empty lines into a single text string
def get_full_text(split):
    return " ".join([t['text'] for t in dataset[split] if t['text'].strip() != ""])

text = get_full_text("train")
words = text.lower().split()

# Build vocabulary
vocab = sorted(set(words))
word2idx = {w: i for i, w in enumerate(vocab)}
idx2word = {i: w for w, i in word2idx.items()}
vocab_size = len(vocab)

print("Vocab size:", vocab_size)
print("Sample tokens:", words[:10])


Generating validation split: 100%|██████████| 3760/3760 [00:00<00:00, 498848.07 examples/s]


Vocab size: 66649
Sample tokens: ['=', 'valkyria', 'chronicles', 'iii', '=', 'senjō', 'no', 'valkyria', '3', ':']


## 🧹 Step 2: Build Input-Output Pairs

We'll use a fixed context window (`block_size`) and train the model to predict the next word.


In [4]:
block_size = 100  # Number of words used as context

def build_dataset(words, word2idx, block_size):
    X, Y = [], []
    for i in range(len(words) - block_size):
        ctx = words[i:i+block_size]
        tgt = words[i+block_size]
        X.append([word2idx[w] for w in ctx])
        Y.append(word2idx[tgt])
    return torch.tensor(X), torch.tensor(Y)

X_train, Y_train = build_dataset(words, word2idx, block_size)


## 🧠 Step 3: Define a Mini GPT-Style Transformer

We implement a basic Transformer block with self-attention and feed-forward layers.


In [5]:
import torch.nn as nn
import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"

class SelfAttention(nn.Module):
    def __init__(self, embed_dim, n_heads):
        super().__init__()
        self.head_dim = embed_dim // n_heads
        self.n_heads = n_heads
        self.q = nn.Linear(embed_dim, embed_dim)
        self.k = nn.Linear(embed_dim, embed_dim)
        self.v = nn.Linear(embed_dim, embed_dim)

        self.dropout = nn.Dropout(0.1)

        self.out = nn.Linear(embed_dim, embed_dim)

    def forward(self, x):
        B, T, C = x.shape
        q = self.q(x).view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        k = self.k(x).view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        v = self.v(x).view(B, T, self.n_heads, self.head_dim).transpose(1, 2)

        att = (q @ k.transpose(-2, -1)) / (self.head_dim ** 0.5)
        mask = torch.tril(torch.ones(T, T, device=x.device)).unsqueeze(0).unsqueeze(0)
        att = att.masked_fill(mask == 0, float('-inf'))
        att = self.dropout(torch.softmax(att, dim=-1))
        out = att @ v
        out = out.transpose(1, 2).contiguous().view(B, T, C)
        return self.out(out)

class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, n_heads):
        super().__init__()
        self.attn = SelfAttention(embed_dim, n_heads)
        self.ln1 = nn.LayerNorm(embed_dim)
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, 4 * embed_dim),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(4 * embed_dim, embed_dim)
        )
        self.ln2 = nn.LayerNorm(embed_dim)

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.ff(self.ln2(x))
        return x

class MiniGPT(nn.Module):
    def __init__(self, vocab_size, block_size, embed_dim=256, n_heads=8, n_layers=6):
        super().__init__()
        self.token_embed = nn.Embedding(vocab_size, embed_dim)
        self.pos_embed = nn.Embedding(block_size, embed_dim)
        self.blocks = nn.Sequential(*[
            TransformerBlock(embed_dim, n_heads) for _ in range(n_layers)
        ])
        self.ln_f = nn.LayerNorm(embed_dim)
        self.fc = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        B, T = x.shape
        tok = self.token_embed(x)
        pos = self.pos_embed(torch.arange(T, device=x.device))
        x = tok + pos
        x = self.blocks(x)
        x = self.ln_f(x)
        return self.fc(x)


## 🏋️ Step 4: Train the MiniGPT Model

We train using cross-entropy loss to predict the next word in the sequence.


In [7]:
model = MiniGPT(vocab_size, block_size=100, embed_dim=256, n_heads=8, n_layers=6).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

X_train, Y_train = X_train.to(device), Y_train.to(device)
batch_size = 64

for step in range(5000):
    idx = torch.randint(0, X_train.size(0), (batch_size,))
    x_batch = X_train[idx]
    y_batch = Y_train[idx]

    logits = model(x_batch)
    logits = logits[:, -1, :]  # only last prediction
    loss = loss_fn(logits, y_batch)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if step % 50 == 0:
        print(f"Step {step}: loss = {loss.item():.4f}")


Step 0: loss = 11.2213
Step 50: loss = 8.9020
Step 100: loss = 7.6960
Step 150: loss = 8.0535
Step 200: loss = 7.2322
Step 250: loss = 7.2231
Step 300: loss = 7.9694
Step 350: loss = 7.8246
Step 400: loss = 8.0342
Step 450: loss = 7.1964
Step 500: loss = 7.2850
Step 550: loss = 7.1472
Step 600: loss = 7.2061
Step 650: loss = 8.4183
Step 700: loss = 6.3419
Step 750: loss = 7.7102
Step 800: loss = 6.5467
Step 850: loss = 7.0880
Step 900: loss = 7.3482
Step 950: loss = 5.8223
Step 1000: loss = 7.1786
Step 1050: loss = 7.2845
Step 1100: loss = 6.9740
Step 1150: loss = 7.3918
Step 1200: loss = 6.7002
Step 1250: loss = 7.1691
Step 1300: loss = 7.0572
Step 1350: loss = 7.2755
Step 1400: loss = 7.4593
Step 1450: loss = 7.8055
Step 1500: loss = 6.5807
Step 1550: loss = 7.4989
Step 1600: loss = 6.4191
Step 1650: loss = 6.7452
Step 1700: loss = 6.8947
Step 1750: loss = 7.0510
Step 1800: loss = 7.1813
Step 1850: loss = 6.8765
Step 1900: loss = 6.9638
Step 1950: loss = 7.2110
Step 2000: loss = 7.42

## 🧪 Step 5: Generate Text

Now we'll use the trained model to generate text word-by-word from a prompt.


In [8]:
def generate(model, start_words, max_new_tokens=20):
    model.eval()
    context = [word2idx.get(w, 0) for w in start_words.lower().split()]
    context = torch.tensor(context[-block_size:], dtype=torch.long).unsqueeze(0).to(device)

    generated = start_words.split()
    for _ in range(max_new_tokens):
        logits = model(context)
        next_logits = logits[0, -1, :]
        probs = torch.softmax(next_logits, dim=-1)
        next_idx = torch.multinomial(probs, num_samples=1).item()
        next_word = idx2word[next_idx]
        generated.append(next_word)
        context = torch.cat([context, torch.tensor([[next_idx]], device=device)], dim=1)
        context = context[:, -block_size:]
    return ' '.join(generated)


In [9]:
print(generate(model, "Why is soccer to good", max_new_tokens=20))


Why is soccer to good replaced asian much enough republican shear barron home one really after 28 negative appointed take clone awarded receive so well
